In [1]:
# Torch
import torch
import torch.optim as optim
from torcheval.metrics import *

import pickle
from captum.attr import *
import random
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# Custom modules
from preprocessing_post_fastsurfer.subject import *
from preprocessing_post_fastsurfer.vis import *
from ozzy_torch_utils.split_dataset import *
from ozzy_torch_utils.subject_dataset import *
from ozzy_torch_utils.plot import *
from ozzy_torch_utils.train_nn import *
from ozzy_torch_utils.model_parameters import *
from ozzy_torch_utils.init_dataloaders import *
from explain_pointnet import *

In [2]:
# Load dataset
data_path = "/uolstore/home/users/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch-disk/full-datasets/hcampus-1.5T-cohort"

subject_list = find_subjects_parallel(data_path)

Csv files: ['/uolstore/home/users/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch-disk/full-datasets/hcampus-1.5T-cohort/idaSearch_3_19_2025.csv']


In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

Checking the significance of left and right hippocampi on models run with both

In [4]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_19-03-2025_16-12-19/run_19-03-2025_16-12-19_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [5]:
subject = sample(subject_list, 1)[0]

cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_Right-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))

In [6]:
attributions, pred_research_group = pointnet_ig(model, cloud, device)

vis_attributions(attributions, subject, cloud, pred_research_group)

Widget(value='<iframe src="http://localhost:38667/index.html?ui=P_0x7fcf5f4028d0_0&reconnect=auto" class="pyvi…

Interesting experiment comparing attributions from two permutations of the same cloud

In [7]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_18-03-2025_15-35-05/run_18-03-2025_15-35-05_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [8]:
cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))
    

In [9]:
attributions_orig, pred_research_group_orig = pointnet_ig(model, cloud, device)

shuffler = np.random.permutation(cloud.shape[0])

unshuffler = np.argsort(shuffler)

cloud_shuffled = np.array([cloud[i] for i in shuffler])

attributions_shuffled, pred_research_group_shuffle = pointnet_ig(model, cloud_shuffled, device)

cloud_unshuffled = np.array([cloud_shuffled[i] for i in unshuffler])

attributions_unshuffled = np.array([attributions_shuffled[i] for i in unshuffler])

attributions_diff = attributions_orig - attributions_unshuffled

print(attributions_diff)

if pred_research_group_orig != pred_research_group_shuffle:
    
    print("Research groups are different after shuffle")
    

[[ 5.49105226e-08 -6.40503466e-08  2.59053946e-07]
 [ 5.38781366e-12  1.37038393e-12  2.20482000e-11]
 [-5.47195059e-08  2.18233865e-08 -1.38176337e-07]
 ...
 [-1.90427179e-15 -6.07153084e-16  2.94021133e-16]
 [ 9.72007618e-08  9.81568395e-09  4.23535125e-08]
 [-2.11567181e-09  6.11278209e-10 -2.69904116e-10]]


In [10]:
vis_attributions(attributions_orig, subject, cloud, pred_research_group_orig)

vis_attributions(attributions_shuffled, subject, cloud_shuffled, pred_research_group_shuffle)

vis_attributions(attributions_unshuffled, subject, cloud_unshuffled, pred_research_group_shuffle)

vis_attributions(attributions_diff, subject, cloud, pred_research_group_orig)

Widget(value='<iframe src="http://localhost:38667/index.html?ui=P_0x7fd040324230_1&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:38667/index.html?ui=P_0x7fcef25ee150_2&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:38667/index.html?ui=P_0x7fce64535520_3&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:38667/index.html?ui=P_0x7fce64537230_4&reconnect=auto" class="pyvi…